# PR9-2 Frequent Itemsets

## Opdracht

PR9-2: Great Outdoors wil graag weten welke producten vaak samen gekocht worden door klanten, door het bouwen van Frequent Itemsets met A-Priori-algoritme. Tip: merge eerst de tabellen 'product' en 'order_details' om een juiste tabel met brongegevens te krijgen waarop je het algoritme kan toepassen. 
- Pas waar nodig Dummy Encoding toe.
- Train het initiële algoritme.
- Experimenteer met meerdere support & confidence thresholds.
- Gebruik [deze webpagina](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6) als inspiratie.

# Implementatie

## Library Imports

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn import metrics
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import sqlite3
import csv
import pyodbc

from apyori import apriori


## Util methods

In [2]:

def horizontal_rule(func) : 
    def wrapped_rule() :
        print( "=" * 30)
        func()
        print( "=" * 30)
    return wrapped_rule

## Data inlezen en samenvoegen

### Databases laden

In [3]:
sales_con = None
crm_con = None
staff_con = None
GO_SALES_PRODUCT_FORECASTData = None
GO_SALES_INVENTORY_LEVELSData = None



try : 
        sales_con = sqlite3.connect("../Assets/Week 2/go_sales.sqlite")
        print("Connected to go_sales sqlite.")


        crm_con = sqlite3.connect("../Assets/Week 2/go_crm.sqlite")
        print("Connected to go_crm sqlite.")


        staff_con = sqlite3.connect("../Assets/Week 2/go_staff.sqlite")
        print("Connected to go_staff sqlite.")

        GO_SALES_PRODUCT_FORECASTData = pd.read_csv("../Assets/Week 2/GO_SALES_PRODUCT_FORECASTData.csv")
        print("Connected to GO_SALES_PRODUCT_FORECASTData")

        GO_SALES_INVENTORY_LEVELSData = pd.read_csv("../Assets/Week 2/GO_SALES_INVENTORY_LEVELSData.csv", header=0, index_col=False)
        print("Connected to GO_SALES_INVENTORY_LEVELSData.csv")


except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)

DataWarehouse_con = None

try : 
        pass
except : 
        pass




Connected to go_sales sqlite.
Connected to go_crm sqlite.
Connected to go_staff sqlite.
Connected to GO_SALES_PRODUCT_FORECASTData
Connected to GO_SALES_INVENTORY_LEVELSData.csv


### Tabellen laden

In [4]:
global sales_branch
sales_branch = pd.read_sql_query("SELECT * FROM {}".format("sales_branch"),sales_con)
sales_branch = sales_branch[sales_branch.columns.drop(list(sales_branch.filter(regex="TRIAL")))]

global retailer_site
retailer_site = pd.read_sql_query("SELECT * FROM {}".format("retailer_site"),sales_con)
retailer_site = retailer_site[retailer_site.columns.drop(list(retailer_site.filter(regex="TRIAL")))]

global country
country = pd.read_sql_query("SELECT * FROM {}".format("country"),sales_con)
country = country[country.columns.drop(list(country.filter(regex="TRIAL")))]

global product
product = pd.read_sql_query("SELECT * FROM {}".format("product"),sales_con)
product = product[product.columns.drop(list(product.filter(regex="TRIAL")))]

global product_type
product_type = pd.read_sql_query("SELECT * FROM {}".format("product_type"),sales_con)
product_type = product_type[product_type.columns.drop(list(product_type.filter(regex="TRIAL")))]

global product_line
product_line = pd.read_sql_query("SELECT * FROM {}".format("product_line"),sales_con)
product_line = product_line[product_line.columns.drop(list(product_line.filter(regex="TRIAL")))]

global order_header
order_header = pd.read_sql_query("SELECT * FROM {}".format("order_header"),sales_con)
order_header = order_header[order_header.columns.drop(list(order_header.filter(regex="TRIAL")))]

global order_details
order_details = pd.read_sql_query("SELECT * FROM {}".format("order_details"),sales_con)
order_details = order_details[order_details.columns.drop(list(order_details.filter(regex="TRIAL")))]


### Dataframe opstellen

In [5]:
dataset = pd.merge(order_details, product, left_on="PRODUCT_NUMBER", how="inner", right_on="PRODUCT_NUMBER")



dataset["ORDER_DETAIL_CODE"] = dataset["ORDER_DETAIL_CODE"].astype(int)
dataset["PRODUCT_NUMBER"] = dataset["PRODUCT_NUMBER"] .astype(int)
dataset["ORDER_NUMBER"] = dataset["ORDER_NUMBER"].astype(int)

display(dataset.dtypes)


display(dataset)

ORDER_DETAIL_CODE     int32
ORDER_NUMBER          int32
PRODUCT_NUMBER        int32
QUANTITY              int64
UNIT_COST            object
UNIT_PRICE           object
UNIT_SALE_PRICE      object
INTRODUCTION_DATE    object
PRODUCT_TYPE_CODE    object
PRODUCTION_COST      object
MARGIN               object
PRODUCT_IMAGE        object
LANGUAGE             object
PRODUCT_NAME         object
DESCRIPTION          object
dtype: object

,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,100000,8462,96,16,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
1,100001,9111,96,20,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
2,100002,8451,96,24,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
3,100003,8453,96,18,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
4,100004,8439,96,20,16.1,22.54,22.54,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99995,8459,95,146,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43059,99996,8457,95,172,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43060,99997,9267,95,192,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."
43061,99998,8441,95,192,2.76,5.55,5.55,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv..."


## Apriori Algoritme

### Data vormgeven

In [6]:
ItemsetList = []

for order in dataset.ORDER_NUMBER.unique() :
    # print(f" ordernumber : {order}")
    
    orderdataframe = dataset.loc[(dataset["ORDER_NUMBER"] == order),:]
    Itemlist = []

    for index, row in orderdataframe.iterrows(): 
        
        Itemlist.append(row["PRODUCT_NAME"])
        # print(Itemlist)
    
    if len(Itemlist) == 1 : 
        pass
    else :
        ItemsetList.append(Itemlist)


print( len(ItemsetList))
print(ItemsetList)


5018
[['Compact Relief Kit', 'Hailstorm Steel Irons', 'Hailstorm Titanium Woods Set', 'Blue Steel Max Putter', 'Course Pro Golf and Tee Set', 'Course Pro Gloves', 'Polar Sun', 'BugShield Lotion Lite', 'BugShield Lotion', 'Sun Shelter 30'], ['Compact Relief Kit', 'Deluxe Family Relief Kit', 'TrailChef Canteen', 'Star Peg', 'Hibernator', 'Hibernator Camp Cot', 'Canyon Mule Climber Backpack', 'Firefly 4', 'EverGlow Single', 'BugShield Natural', 'Sun Blocker'], ['Compact Relief Kit', 'Deluxe Family Relief Kit', 'Hailstorm Titanium Woods Set', 'Lady Hailstorm Steel Woods Set', 'Canyon Mule Journey Backpack', 'Mountain Man Analog', 'BugShield Lotion', 'Sun Shelter 30', 'Sun Shield'], ['Compact Relief Kit', 'Aloe Relief', 'Lady Hailstorm Titanium Irons', 'Course Pro Golf Bag', 'Mountain Man Digital', 'BugShield Spray', 'Sun Shelter 15'], ['Compact Relief Kit', 'Deluxe Family Relief Kit', 'Calamine Relief', 'TrailChef Kitchen Kit', 'TrailChef Double Flame', 'TrailChef Kettle', 'TrailChef Utens

In [14]:
association_rules = apriori(ItemsetList, min_support=0.02, min_confidence=0.20, min_lift=3, min_length=4)
results = list(association_rules)

print( len(results))

display(results[54])

resultsframe = pd.DataFrame(results)
display(resultsframe)
# for rule in results : 
#     print(f"{results[rule]}\n{"="*30}")

81


RelationRecord(items=frozenset({'Granite Grip', 'Granite Carabiner'}), support=0.021123953766440814, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Granite Carabiner'}), items_add=frozenset({'Granite Grip'}), confidence=0.3212121212121212, lift=3.8286043331173976), OrderedStatistic(items_base=frozenset({'Granite Grip'}), items_add=frozenset({'Granite Carabiner'}), confidence=0.2517814726840855, lift=3.828604333117397)])

,items,support,ordered_statistics
0,"(Blue Steel Max Putter, Blue Steel Putter)",0.021523,"[((Blue Steel Max Putter), (Blue Steel Putter)..."
1,"(Blue Steel Max Putter, Course Pro Golf Bag)",0.020128,"[((Blue Steel Max Putter), (Course Pro Golf Ba..."
2,"(Hailstorm Titanium Irons, Blue Steel Max Putter)",0.027501,"[((Blue Steel Max Putter), (Hailstorm Titanium..."
3,"(Blue Steel Max Putter, Hailstorm Titanium Woo...",0.023515,"[((Blue Steel Max Putter), (Hailstorm Titanium..."
4,"(Blue Steel Max Putter, Lady Hailstorm Titaniu...",0.020925,"[((Blue Steel Max Putter), (Lady Hailstorm Tit..."
...,...,...,...
76,"(Hailstorm Titanium Irons, Lady Hailstorm Stee...",0.020128,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
77,"(Hailstorm Titanium Irons, Lady Hailstorm Tita...",0.024312,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
78,"(Hailstorm Titanium Irons, Lady Hailstorm Tita...",0.025707,"[((Hailstorm Titanium Irons), (Lady Hailstorm ..."
79,"(Hailstorm Titanium Woods Set, Lady Hailstorm ...",0.022320,"[((Hailstorm Titanium Woods Set), (Lady Hailst..."


RelationRecord(
    items=frozenset(
        {'Blue Steel Max Putter', 'Blue Steel Putter'}), 
        support=0.021522518931845355, 
        ordered_statistics=[OrderedStatistic
                                (items_base=frozenset({'Blue Steel Max Putter'}),
                                 items_add=frozenset({'Blue Steel Putter'}), 
                                 confidence=0.304225352112676, 
                                 lift=3.9345433425294023), 
                            OrderedStatistic(
                                items_base=frozenset({'Blue Steel Putter'}), 
                                items_add=frozenset({'Blue Steel Max Putter'}), 
                                confidence=0.2783505154639175, 
                                lift=3.9345433425294023)
                                ]
                                )